In [50]:
import pandas as pd
import utils 
import plotly.graph_objects as go 

In [51]:
pair = "EUR_USD"
granularity = 'H1'

In [52]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity) )

In [53]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2023-12-06T08:00:00.000000000Z,5540,1.07898,1.07904,1.07750,1.07868,1.07890,1.07897,1.07743,1.07860,1.07906,1.07911,1.07757,1.07875
1,2023-12-06T09:00:00.000000000Z,5148,1.07869,1.07979,1.07860,1.07911,1.07862,1.07972,1.07853,1.07903,1.07876,1.07986,1.07867,1.07919
2,2023-12-06T10:00:00.000000000Z,4071,1.07912,1.07960,1.07874,1.07911,1.07905,1.07953,1.07867,1.07904,1.07920,1.07968,1.07882,1.07918
3,2023-12-06T11:00:00.000000000Z,3709,1.07909,1.07909,1.07780,1.07800,1.07901,1.07902,1.07773,1.07792,1.07917,1.07917,1.07788,1.07807
4,2023-12-06T12:00:00.000000000Z,3832,1.07800,1.07808,1.07724,1.07774,1.07793,1.07800,1.07717,1.07766,1.07807,1.07816,1.07731,1.07781


In [54]:
df.describe()

,volume
count,3999.000000
mean,3050.187547
std,2248.804381
min,95.000000
25%,1546.500000
50%,2532.000000
75%,3896.000000
max,21907.000000


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    3999 non-null   object
 1   volume  3999 non-null   int64 
 2   mid_o   3999 non-null   object
 3   mid_h   3999 non-null   object
 4   mid_l   3999 non-null   object
 5   mid_c   3999 non-null   object
 6   bid_o   3999 non-null   object
 7   bid_h   3999 non-null   object
 8   bid_l   3999 non-null   object
 9   bid_c   3999 non-null   object
 10  ask_o   3999 non-null   object
 11  ask_h   3999 non-null   object
 12  ask_l   3999 non-null   object
 13  ask_c   3999 non-null   object
dtypes: int64(1), object(13)
memory usage: 437.5+ KB


In [56]:
non_cols =['time', 'volume']

In [57]:
mod_cols = [x for x in df.columns if x not in non_cols]

set mod_cols = to a list and get us each value in the  dataframe column if that 'x' is not in the list, then create a new list 
this happens often that we need to convert strings to numbers or any other data type 

In [58]:
mod_cols

['mid_o',
 'mid_h',
 'mid_l',
 'mid_c',
 'bid_o',
 'bid_h',
 'bid_l',
 'bid_c',
 'ask_o',
 'ask_h',
 'ask_l',
 'ask_c']

In [59]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3999 non-null   object 
 1   volume  3999 non-null   int64  
 2   mid_o   3999 non-null   float64
 3   mid_h   3999 non-null   float64
 4   mid_l   3999 non-null   float64
 5   mid_c   3999 non-null   float64
 6   bid_o   3999 non-null   float64
 7   bid_h   3999 non-null   float64
 8   bid_l   3999 non-null   float64
 9   bid_c   3999 non-null   float64
 10  ask_o   3999 non-null   float64
 11  ask_h   3999 non-null   float64
 12  ask_l   3999 non-null   float64
 13  ask_c   3999 non-null   float64
dtypes: float64(12), int64(1), object(1)
memory usage: 437.5+ KB


In [61]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000
mean,3050.187547,1.082687,1.083188,1.082185,1.082688,1.082603,1.083110,1.082101,1.082610,1.082771,1.083270,1.082264,1.082767
std,2248.804381,0.009110,0.009119,0.009089,0.009110,0.009109,0.009118,0.009087,0.009109,0.009111,0.009120,0.009091,0.009111
min,95.000000,1.061000,1.061260,1.060110,1.061010,1.060920,1.061190,1.060040,1.060940,1.061070,1.061330,1.060180,1.061080
25%,1546.500000,1.076380,1.076820,1.075875,1.076380,1.076300,1.076745,1.075790,1.076300,1.076450,1.076895,1.075990,1.076465
50%,2532.000000,1.083120,1.083680,1.082620,1.083140,1.083040,1.083600,1.082550,1.083060,1.083200,1.083750,1.082700,1.083220
75%,3896.000000,1.088445,1.088865,1.087930,1.088425,1.088345,1.088785,1.087850,1.088340,1.088530,1.088955,1.088005,1.088500
max,21907.000000,1.112920,1.113960,1.112270,1.112940,1.112840,1.113880,1.112190,1.112860,1.112990,1.114040,1.112350,1.113020


In [62]:
df_plot = df.iloc[-100:]

In [63]:
df_plot.shape

(100, 14)

In [64]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor= '#24A06B',
    decreasing_fillcolor= '#CC2E3C',
    increasing_line_color= '#2EC886',
    decreasing_line_color= '#FF3A4C'
))
fig.update_layout(width=1300, height=400,
    margin=dict(l=10, r=10, b=10, t=10),
    font=dict(size=10, color='#e1e1e1'),             
    paper_bgcolor='#1e1e1e',
    plot_bgcolor='#1e1e1e')
fig.update_xaxes(
    gridcolor='#1f292f',
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor='#1f292f',
    showgrid=True
)                  

fig.show()

In [65]:
df_ma =df 